# Working with MPAS Meshes

##  Mesh Definition

All components of MPAS lie on a C-grid staggered Voronoi Mesh as seen in the figure below:

<p align="center">
  <img src="../_static/examples/mpas/c-grid.png"
  width="400" / >
</p>

This type of mesh is represented through two different sub-meshes. The Primal Mesh represents the Voronoi Mesh and the Dual Mesh represents the Triangular Mesh.

*to-do: more in-depth description/overview of MPAS from the website and specification document.*




## Imports

In [ ]:
import xarray as xr
import numpy as np
import uxarray as ux

## Grid and Data Files

As mentioned in earlier notebooks, the grid definition and data variables are typically stored in separate files. However, in this example, our dataset will contain both within the same file, which is often the case when working with smaller datasets. With this in mind, we will be working exclusively with 'Grid' objects and not with 'UxDataset` or `UxDataArray` objects.



In [ ]:
mpas_root_filepath = "../../test/meshfiles/mpas/"
mpas_dataset_filepath = mpas_root_filepath + "QU/mesh.QU.1920km.151026.nc"

xrds_mpas = xr.open_dataset(mpas_dataset_filepath)
xrds_mpas

This dataset contains the full set of connectivity variables needed to describe an MPAS mesh, as described in the MPAS Specification Document [2]. Below is a list of connectivity variables and their geometric meaning, which will be further described when comparing to the UGRID encoding:


### Primal Mesh
* **lonVertex, latVertex**: Corner Vertices of Primal Mesh cells
* **lonCell, latCell**: Center Vertices of Primal Mesh cells
* **verticesOnCell**: Vertex indices that surround each Primal Mesh cell
* **verticesOnEdge**: Vertex indices that saddle a given edge
* **nEdgesOnCell**: Maximum number of edges that can surround a cell

### Dual Mesh
* **lonCell, latCell**: Corner Vertices of Dual Mesh cells
* **lonVertex, latVertex**: Center Vertices of Dual Mesh cells
* **cellsOnVertex**: Vertex indices that surround each Dual Mesh cell
* **cellsOnEdge**: Vertex indices that saddle a given edge


## Constructing a Grid Object

The `xarray.Dataset` that we opened above stores the coordinates and connectivity variables for both the Primal and Dual meshes together in a single dataset. These variables are stored according to the MPAS Specification standards. With UXarray, we can create `Grid` objects for either the Primal or Dual meshes. The `uxarray.Grid` class parses our input dataset, detects that it is represented in the MPAS specifications, and encodes either the Primal or Dual mesh in the UGRID conventions.

In [ ]:
primal_mesh = ux.open_grid(mpas_dataset_filepath, use_dual=False)
dual_mesh = ux.open_grid(mpas_dataset_filepath, use_dual=True)

In [ ]:
primal_mesh

In [ ]:
dual_mesh

## Relationship between MPAS and UGRID
*to-do: describe MPAS to UGRID encoding, differences in fill values, standardized datatypes, and how geometric connectivity variables relate between these two conventions*

*create a table to showcase the mapping between MPAS and UGRID*

### Connectivity (Primal Mesh)
`Mesh2_node_x` & `Mesh2_node_y`
* Derived from `lonVertex` & `latVertex`
* Converted from Radians to Degrees

`Mesh2_face_nodes`
* Derived from `verticesOnCell`
* Padding is replaced with `INT_FILL_VALUE`
* Missing Values (zeros) replaced with `INT_FILL_VALUE`
* Converted to zero-index

## Functionality

### Face Area

In [ ]:
primal_mesh_total_area = uxds_primal.uxgrid.calculate_total_face_area()
primal_mesh_total_area

In [ ]:
primal_mesh_face_areas = uxds_primal.uxgrid.face_areas
primal_mesh_face_areas.sum()

In [ ]:
dual_mesh_total_area = uxds_dual.uxgrid.calculate_total_face_area()
dual_mesh_total_area

In [ ]:
dual_mesh_face_areas = uxds_dual.uxgrid.face_areas
primal_mesh_face_areas.sum()

### Integration

In [ ]:
# to-do, need to fix function before adding

## Visualization

*very messy, waiting on other PR's to get merged before doing primal mesh*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection

from uxarray.utils.helpers import close_face_nodes

### Helper Functions

In [ ]:
def get_verts(uxds):
    nodes_per_face = uxds.uxgrid.nNodes_per_face.values
    x = uxds.uxgrid.Mesh2_node_x.values
    y = uxds.uxgrid.Mesh2_node_y.values
    closed_face_nodes = close_face_nodes(uxds.uxgrid.Mesh2_face_nodes.values,
                                         uxds.uxgrid.nMesh2_face,
                                         uxds.uxgrid.nMaxMesh2_face_nodes)

    verts = []
    for face_node, n_nodes in zip(closed_face_nodes, nodes_per_face):
        polygon_x = x[face_node[0:n_nodes]]
        polygon_y = y[face_node[0:n_nodes]]

        if np.any(np.abs(polygon_x - 180) > 170):
            continue

        vert = np.array([polygon_x, polygon_y])
        verts.append(vert.T)

    return verts

### Primal Mesh

In [ ]:
primal_vertices = get_verts(uxds_primal)
primal_polygons = PolyCollection(primal_vertices, edgecolors='Black', alpha=0.5)

fig, ax = plt.subplots()
ax.add_collection(primal_polygons, autolim=False)
ax.set_xlim(0, 360)
ax.set_ylim(-80, 80)
ax.set_title("Primal Mesh")

### Dual Mesh

In [ ]:
dual_vertices = get_verts(uxds_dual)
dual_polygons = PolyCollection(dual_vertices, edgecolors='Black', alpha=0.5)

fig, ax = plt.subplots()
ax.add_collection(dual_polygons, autolim=False)
ax.set_xlim(0, 360)
ax.set_ylim(-80, 80)
ax.set_title("Dual Mesh")

## Primal and Dual

In [ ]:
dual_vertices = get_verts(uxds_dual)
dual_polygons = PolyCollection(dual_vertices, edgecolors='Black', alpha=0.5)
primal_vertices = get_verts(uxds_primal)
primal_polygons = PolyCollection(primal_vertices, edgecolors='Black', alpha=0.5)

fig, ax = plt.subplots()
ax.add_collection(primal_polygons, autolim=False)
ax.add_collection(dual_polygons, autolim=False)
ax.set_xlim(0, 360)
ax.set_ylim(-80, 80)
ax.set_title("Dual & Primal Mesh")

## References

[1] https://mpas-dev.github.io/

[2] https://mpas-dev.github.io/files/documents/MPAS-MeshSpec.pdf

[3] http://ugrid-conventions.github.io/ugrid-conventions/